In [3]:
from helpers import pdf_to_chunks, embed_send_to_db, list_files
import os
import shutil
import pandas as pd

source_path = file_path = os.path.join("data", "files_before_processing")
destination_path = file_path = os.path.join("data", "files_post_processing")

# set metadata for every file in the source folder (will be overwritten by the CSV if a match is found)
metadata_global = {"type": "localplan"}

# get metadata per file from the CSV
file_path = os.path.join("data", "metadata.csv")
if os.path.exists(file_path):
    metadata_csv_exists = True
    metadata_df = pd.read_csv(file_path)
else:
    metadata_csv_exists = False

# start processing the pdfs
pdfs = list_files(source_path)

for pdf in pdfs:
    print(f"Processing {pdf}")

    pdf_path = os.path.join(source_path, pdf)
    try:
        chunks = pdf_to_chunks(pdf_path)

        if metadata_csv_exists:
            pdf_metadata = metadata_df.loc[metadata_df["filename"] == pdf].to_dict(
                orient="records"
            )[0]

            metadata = metadata_global | pdf_metadata

        else:
            metadata = metadata_global
        print(metadata)

        embed_send_to_db(chunks, pdf, metadata)

        print("*** COMPLETE ***")

        shutil.move(
            f"data/files_before_processing/{pdf}", f"data/files_post_processing/")
    except Exception as e:
        shutil.move(
            f"data/files_before_processing/{pdf}", f"data/files_failed_processing/")
        print(f"Error processing {pdf}: {e}")
        continue

Processing Arun_District_Council.pdf
